# Foundations of Machine Learning - Session 05

- *Course*: Foundations of Machine Learning
- *Session*: 04
- *Unit*: Classification Trees

This notebook develops tree-based classifiers using Numpy. Both the ID3 and CART algorithm will be constructed and their results compared.

In [10]:
import numpy as np
from collections import Counter # Counter(["a", "b", "a"]) == {"a": 2, "b":1}

## Data

For this notebooks' classifiction task, we are going to use the famous Iris toy dataset. The data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant. Each sample consist of four features, each describing one property of the flower: `sepal_width`, `sepal_length`, `petal_width`, `petal_length`. One class is linearly separable from the other 2; the latter are NOT linearly separable from each other.

The code below is pre-filled to load the Iris dataset using `sklearn`, and randomly split it into 80:20 train/test subset.

In [11]:
from sklearn.datasets import load_iris
# Load Iris dataset from sklearn
data = load_iris()
# Assemble D
D = list(zip(
    data.data,
    data.target
))
# Shuffle D
np.random.shuffle(D)
# 80:20 split into train and test set
D_train = D[:int(len(D) * 0.8)]
D_test = D[int(len(D) * 0.8):]

Since we want to train both CART and ID3, one problem arises: ID3 can only deal with categorical data, yet the Iris dataset features numerical values. We therefore need to *quantize* the data.

**Exercise**: quantize each feature separately by representing each value by the index of its quartile, i.e. a value in the 4th quartile is represented by `3`. Then, quantize the train and test sets to form `D_train_quant` and `D_test_quant`.

*Hints*:
- use `np.quantile` to find the quartiles for each feature
- use `np.digitize` to find the correct quartile for each value
- `np.apply_along_axis` allows you to apply a function separately to each axis of an array.

In [12]:
def quantize(arr, q=4):
    """
    Quantizes the input array into the specified number of quantiles.
    :param arr: array to quantize
    :param q: number of quantiles to quantize with
    :return: quantized array
    """
    bins = np.quantile(arr, np.linspace(0, 1, q+1)[1:-1])
    return np.digitize(arr, bins)

# Quantize data
D_quant = list(zip(
    np.apply_along_axis(quantize, 0, data.data),
    data.target
))

# Shuffle D_quant
np.random.shuffle(D_quant)
# 80:20 split into quantized train and test set
D_quant_train = D_quant[:int(len(D) * 0.8)]
D_quant_test = D_quant[int(len(D) * 0.8):]

## Entropy

For both ID3 and CART, we will use entropy as splitting criterion. It is given as:

$$ \iota(D) = -\sum_{i=1}^k \frac{|\{\mathbf{x}, c_i)\in D\}|}{|D|}\cdot\log_2\frac{|\{\mathbf{x}, c_i)\in D\}|}{|D|} $$

**Exercise**: implement a function to calculate the entropy of a given dataset D.

*Hints*:
- use `Counter` to get the number of instances for each class.

In [13]:
def entropy(D):
    """
    Computes the entropy for a given set of data
    :param D: the dataset to compute entropy for. Consists of (feature_vector, class) tuples.
    :return:
    """
    # Compute total number of samples
    n_samples = len(D)
    # Count class occurrences
    class_counts = Counter([c for _, c in D])
    # Transform class counts into probabilities
    class_counts = {k: v/n_samples for k,v in class_counts.items()}
    # Compute entropy of complete Dataset
    h = -sum(class_counts[k] * np.log2(class_counts[k]) for k in class_counts.keys())
    return h

## ID3

The first tree algorithm we will implement is ID3. The implementation is analogous to the lecture and split into the actual ID3 algorithm `train_id3`, and the information gain function that returns the entropy delta for a given feature on D (`information_gain_id3`). Information gain is formulated as:

$$\Delta\iota = \iota(D) - \sum_{l=1}^m\frac{|D_l|}{D}\cdot\iota(D_l) $$



**Exercise** implement `information_gain_id3`, which calculates the information gain of D and a give feature.

In [14]:
def information_gain_id3(D, feature):
    """
    Computes the information gain of feature f on dataset D.
    :param D: dataset to compute the information gain with.
    :param feature: feature to compute the information gain for.
    :return: the information gain.
    """
    # Compute total entropy
    h_t = entropy(D)
    # Compute feature expressions
    F = Counter([d[0][feature] for d in D])
    # Add weighted entropy of each feature expression to total entropy
    h_f = 0
    for f in F.keys():
        h_f += F[feature]/len(D) * -entropy(list(filter(lambda x: x[0][feature] == f, D)))
    return h_t - h_f

Next, we will implement the (recursive) ID3 training function. The tree itself is to be represented by nested dictionaries. Each has three keys:
1. `label` – specifies the label of this tree node
2. `feature` - specifies the feature that is used for splitting at this node
3. `children` - contains a nested dict, where keys are the feature values of the splitting feature, and values are the subtrees (dictionaries) for each.

The `train_id3` function is recursive: the `children` of a node are determined by recursively applying `train_id3` on the reduced data- and feature set. Don't forget the base case and exit criteria as specified in the lecture, to avoid infinite recursion!

**Exercise**: implement `train_id3`.

*Hints*:
- leave the `feature` and `children` keys of the node dictionary at `None`  for leaf nodes
- use `np.argmax` to find the index of the maximum element in a list/array
- you can use set comprehensions to calculate the domain of a feature
- you can use either list comprehensions of `filter()` to build the data subsets for recursion

In [15]:

def train_id3(D, features: list):
    # Create a new node
    t = {
        "label": None,
        "feature": None,
        "children": {}
    }
    # Count the occurrences of each class in D
    class_counts = Counter([c for _, c in D])
    # Find the most common class in D and assign it to t
    t["label"] = max(class_counts, key=class_counts.get)
    # If D is pure, return t
    if len(class_counts) == 1:
        return t
    # No features are left
    if not features:
        return t
    # Compute feature with maximum information gain
    idx_max_f = np.argmax(list(map(lambda f: information_gain_id3(D, f), features)))
    t["feature"] = features[idx_max_f]
    # Remove max feature from feature set
    features.remove(t["feature"])
    # Calculate domain of max feature on D
    dom_A = {d[0][t["feature"]] for d in D}
    # Calculate subtrees for every feature expression
    for a in dom_A:
        # Subset data
        D_a = list(filter(lambda x: x[0][t["feature"]] == a, D))
        # If no data is left
        if len(D_a) > 0:
            # Recursive call
            t["children"][a] = train_id3(D_a, features)
    return t

**Exercise**: apply `train_id3` to `D_train_quant` and all classes (`[0,1,2]`).

In [16]:
id3_model = train_id3(D_quant_train, [0,1,2])

To predict a class for new data using the tree inferred with ID3, we need to write a function that traverses the tree until it reaches a leaf node, i.e. class decision.

**Exercise** implement `predict_id3`, which should traverse the decision tree and follow each node's decision rule given the input data.

*Hints*:
- the implementation should be recursive
- leaf nodes can be identified by checking if `feature is None`, i.e., a node has no decision rule

In [17]:
def predict_id3(model, d):
    """
    Predict a class for a given sample using the given ID3 model.
    :param model: an ID3 decision tree model
    :param d: a feature vector
    :return: the class prediction
    """
    feature = model["feature"]
    if feature is None:
        return model["label"]
    else:
        return predict_id3(model["children"][d[feature]], d)

Like in previous weeks, apply either your own or `sklearns` implementation of evaluation functions to evaluate the effectiveness of your ID3 implementation.

**Exercise** evaluate the ID3 model.

*Hints*:
- pay attention to use the correct train/test splits
- use the quantized version of both train and test data

In [18]:
from sklearn.metrics import classification_report

print(classification_report([d[1] for d in D_quant_test], [predict_id3(id3_model, d[0]) for d in D_quant_test]))

              precision    recall  f1-score   support

           0       1.00      0.58      0.74        12
           1       0.62      0.71      0.67         7
           2       0.53      0.73      0.62        11

    accuracy                           0.67        30
   macro avg       0.72      0.67      0.67        30
weighted avg       0.74      0.67      0.68        30



## CART

Next, implement the CART algorithm. As for ID3, we will implement it in two steps: one function to calculate the information gain (`information_gain_cart`) and one for the actual training loop (`train_cart`). CART differs from ID3 in that it will figure out an optimal decision boundary at each node, thus  the information gain function should not only return the entropy delta, but also the optimal threshold for the specified feature.

The optimal threshold is determined by the following formula:

$$ \Delta\iota(D(t), \{D(t_L), D(t_R)\} = \iota(D(t)) - \frac{|D(t_L)|}{|D|} \cdot \iota(D(t_L)) - \frac{|D(t_R|}{|D|} \cdot \iota(D(t_R))$$

**Exercise**: implement `information_gain_cart`.

*Hints*:
- use the `entropy` function defined previously, applying it to the relevant subsets (left/right) of each split
- calculate the entropy for each possible threshold to identify the optimal one

In [19]:
def information_gain_cart(D, feature):
    """
    Computes the information gain of feature f on dataset D by finding and optimal splitting.
    :param D: dataset to compute the information gain with.
    :param feature: feature to compute the information gain for.
    :return: the information gain and the threshold
    """
    # Compute total entropy
    h_t = entropy(D)
    # Compute splittings
    values = sorted(list(set([d[0][feature] for d in D])))
    splits = [0.5 * (values[i] + values[i+1]) for i in range(len(values) - 1)]

    # Find threshold with maximum information gain
    h_max, threshold = 0, 0
    for split in splits:
        # Build synthetic data labels for each side of the split
        D_l = list(filter(lambda x: x[0][feature] < split, D))
        D_r = list(filter(lambda x: x[0][feature] > split, D))
        h = h_t - (len(D_l)/len(D) * entropy(D_l)) - (len(D_r)/len(D) * entropy(D_r))
        if h > h_max:
            h_max = h
            threshold = split
    # Return delta
    return h_max, threshold

Given the modified information gain formula, the training function also has to be modified. Make three important changes to transform ID3 into CART:
1. the tree representation is different to ID3. The dictionary now contains 5 keys; `label` and `feature` stay the same as before, `threshold` specifies the numerical decision boundary in a node, and `right child`/`left_child` the two subtrees.
2. the information gain function also returns the threshold. Remember to track not only the optimal feature, but also its threshold and persist it in the node dictionary.
3. CART is a binary tree, i.e. the recursion is applied once to the left (feature lower than threshold) and right (feature higher than threshold) data subset. Each is a separate recursion call and is to be persisted in the node dictionary.

**Exercise**: implement `train_cart`

In [20]:
def train_cart(D):
    # Create a new node (
    t = {
        "label": None,
        "feature": None,
        "threshold": 0,
        "left_child": None,
        "right_child": None
    }
    # Compute the set of features
    features = [0,1,2]
    # Count the occurrences of each class in D
    class_counts = Counter([c for _, c in D])
    # Find the most common class in D and assign it to t
    t["label"] = max(class_counts, key=class_counts.get)
    # If D is pure, return t
    if len(class_counts) == 1:
        return t
    # No features are left
    if not features:
        return t

    # Compute max feature and threshold
    splits = [information_gain_cart(D, f) for f in features]
    entropies = [x[0] for x in splits]
    thresholds = [x[1] for x in splits]
    idx_max_f = np.argmax(entropies)
    t["feature"] = features[idx_max_f]
    t["threshold"] = thresholds[idx_max_f]

    # Calculate subtrees for left side of the threshold
    D_l = list(filter(lambda x: x[0][t["feature"]] < t["threshold"], D))
    if len(D_l) > 0:
        t["left_child"] = train_cart(D_l)

    # Calculate subtrees for right side of the threshold
    D_r = list(filter(lambda x: x[0][t["feature"]] > t["threshold"], D))
    if len(D_r) > 0:
        t["right_child"] = train_cart(D_r)

    return t

**Exercise** train a CART model by applying `train_cart` to `D_train`.

*Hint*
- use the unquantized version of the training data
- remember to use only the train split

In [21]:
cart_model = train_cart(D_train)

**Exercise**: implement `predict_cart` to traverse the CART tree and assign a class to data samples.

*Hints*:
- implementation is similar to `predict_id3`
- for the specified feature, determine whether to traverse the right or left subtree based on the decision threshold (left: <, right: >)

In [22]:
def predict_cart(model, d):
    """
    Predict a class for a given sample using the CART given model.
    :param model: an ID3 decision tree model
    :param d: a feature vector
    :return: the class prediction
    """
    feature = model["feature"]
    if feature is None:
        return model["label"]
    if d[feature] < model["threshold"]:
        return predict_cart(model["left_child"], d)
    else:
        return predict_cart(model["right_child"], d)

**Exercise**: evaluate CART on `D_test`.

In [23]:
from sklearn.metrics import classification_report

print(classification_report([d[1] for d in D_test], [predict_cart(cart_model, d[0]) for d in D_test]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       0.90      1.00      0.95         9
           2       1.00      0.92      0.96        13

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



**Exercise**: compare the evaluation results of ID3 and CART. Guess why CART performs better.